<img src="https://github.com/DrPBaksh/image_processing/blob/main/logos/logo_2.png?raw=true" alt="Corndel" width ="301.5" align="left">


# Image Classifiers using Gradio and Hugging Face

1. Import necessary libraries including gradio, torch, clip, and PIL.
2. Load the CLIP model and specify the device (CPU or GPU) to use. Pre trained image classifier
3. Define a function classify_image that takes an input image and a list of categories, preprocesses the image, encodes the image and text using the 4 4. CLIP model, and returns a list of labels and scores as results.
5. Define the inputs and outputs for the Gradio interface using the gr.inputs and gr.outputs functions.
6. Define the title, description, and examples for the Gradio interface.
7. Download example images from GitHub using the requests library and save them locally.
8. Launch the Gradio interface using the gr.Interface function and the launch method, specifying that the interface should be shareable.

In [10]:
# !pip install openai_clip
# 

In [13]:
import gradio as gr
import torch
import clip
from PIL import Image
import numpy as np 
import requests


# Load the CLIP model and preprocess
device = "cuda" if torch.cuda.is_available() else "cpu"
model, preprocess = clip.load("ViT-B/32", device=device)

# Get example images

urls = ['https://github.com/DrPBaksh/workshop-data/raw/main/parrot.jpg', 'https://github.com/DrPBaksh/workshop-data/raw/main/cat.jpg']
print('downloading examples')
for i in range(len(urls)):
    url = urls[i]
    response = requests.get(url)
    with open(urls[i].split('/')[-1], 'wb') as f:
        f.write(response.content)
print('finished downloading examples')


# Define the Gradio interface
def classify_image(input_image, categories):
    # Prepare the image
    input_image = Image.fromarray(np.uint8(input_image))
    input_image = input_image.convert("RGB")
    input_image = preprocess(input_image).unsqueeze(0).to(device)

    # Prepare the candidate labels
    candidate_labels = categories.split(", ")
    text = clip.tokenize(candidate_labels).to(device)

    # Make a prediction using the classifier pipeline
    with torch.no_grad():
        image_features = model.encode_image(input_image)
        text_features = model.encode_text(text)
        logits_per_image = torch.matmul(image_features, text_features.t())
        probs = logits_per_image.softmax(dim=-1)

    # Format the prediction results as a list of labels and scores
    results = []
    for i, (label, prob) in enumerate(zip(candidate_labels, probs[0])):
        results.append("{}: {:.2f}%".format(label, prob.item() * 100))

    # Return the formatted results
    return results

inputs = [
    gr.inputs.Image(label="Input Image"),
    gr.inputs.Textbox(label="Categories", default="dog, cat, bird")
]
outputs = gr.outputs.Textbox(label="Image Classification Results")
title = "Zero-Shot Image Classifier"
description = "Upload an image and select the categories you want to test the image against using a zero-shot learning model."
examples = [
    ["parrot.jpg", "bird"],
    ["cat.jpg", "dog, cat"]
]

gr.Interface(fn=classify_image, inputs=inputs, outputs=outputs, title=title, description=description, examples=examples).launch(share = True)


downloading examples
finished downloading examples


/home/pete/Documents/python_code/hugginFace/venv_hugginFace/lib/python3.8/site-packages/gradio/inputs.py:257: UserWarning: Usage of gradio.inputs is deprecated, and will not be supported in the future, please import your component from gradio.components
  warnings.warn(
/home/pete/Documents/python_code/hugginFace/venv_hugginFace/lib/python3.8/site-packages/gradio/deprecation.py:40: UserWarning: `optional` parameter is deprecated, and it has no effect
  warnings.warn(value)
/home/pete/Documents/python_code/hugginFace/venv_hugginFace/lib/python3.8/site-packages/gradio/inputs.py:27: UserWarning: Usage of gradio.inputs is deprecated, and will not be supported in the future, please import your component from gradio.components
  warnings.warn(
/home/pete/Documents/python_code/hugginFace/venv_hugginFace/lib/python3.8/site-packages/gradio/deprecation.py:40: UserWarning: `numeric` parameter is deprecated, and it has no effect
  warnings.warn(value)
/home/pete/Documents/python_code/hugginFace/ve

Running on local URL:  http://127.0.0.1:7868
Running on public URL: https://a5fcada115dce4baef.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades (NEW!), check out Spaces: https://huggingface.co/spaces


# Image Classifiers using Gradio and Hugging Face

The code below is an image classifier where you do not need to enter in the categoires. It uses the already divided categories from imagenet to categorise your image 


In [49]:
!wget https://raw.githubusercontent.com/openai/CLIP/main/CLIP/imagenet_classes.txt

--2023-04-09 07:58:18--  https://raw.githubusercontent.com/openai/CLIP/main/CLIP/imagenet_classes.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.111.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 404 Not Found
2023-04-09 07:58:18 ERROR 404: Not Found.



In [20]:
import gradio as gr
import torch
import clip
from PIL import Image
import numpy as np

# Load the CLIP model and preprocess
device = "cuda" if torch.cuda.is_available() else "cpu"
model, preprocess = clip.load("ViT-B/32", device=device)

urls = ['https://github.com/DrPBaksh/workshop-data/raw/main/parrot.jpg', 'https://github.com/DrPBaksh/workshop-data/raw/main/cat.jpg']
print('downloading examples')
for i in range(len(urls)):
    url = urls[i]
    response = requests.get(url)
    with open(urls[i].split('/')[-1], 'wb') as f:
        f.write(response.content)
print('finished downloading examples')



# Get all 1000 ImageNet categories
with open("imagenet_Classes.txt") as f:
    candidate_labels = f.readlines()
candidate_labels = [label.strip() for label in candidate_labels]

# Define the Gradio interface
def classify_image(input_image):
    # Prepare the image
    input_image = Image.fromarray(np.uint8(input_image))
    input_image = input_image.convert("RGB")
    input_image = preprocess(input_image).unsqueeze(0).to(device)

    # Prepare the candidate labels
    text = clip.tokenize(candidate_labels).to(device)

    # Make a prediction using the classifier pipeline
    with torch.no_grad():
        image_features = model.encode_image(input_image)
        text_features = model.encode_text(text)
        logits_per_image = torch.matmul(image_features, text_features.t())
        probs = logits_per_image.softmax(dim=-1)

    # Get the top 3 labels and probabilities
    top_k = torch.topk(probs, k=3)
    top_probs = top_k.values[0]
    top_labels = [candidate_labels[i] for i in top_k.indices[0]]

    # Format the prediction results as a list of labels and scores
    results = []
    for i in range(len(top_labels)):
        results.append("{}: {:.2f}%".format(top_labels[i], top_probs[i].item() * 100))

    # Return the formatted results
    return results


inputs = gr.inputs.Image(label="Input Image")
outputs = gr.outputs.Textbox(label="Image Classification Results")
title = "Zero-Shot Image Classifier"
description = "Upload an image and we'll predict its category using a zero-shot learning model trained on the ImageNet dataset."
examples = [["cat.jpg", "parrot.jpg"]]

gr.Interface(fn=classify_image, inputs=inputs, outputs=outputs, title=title, description=description, examples=examples).launch()


downloading examples
finished downloading examples


/home/pete/Documents/python_code/hugginFace/venv_hugginFace/lib/python3.8/site-packages/gradio/inputs.py:257: UserWarning: Usage of gradio.inputs is deprecated, and will not be supported in the future, please import your component from gradio.components
  warnings.warn(
/home/pete/Documents/python_code/hugginFace/venv_hugginFace/lib/python3.8/site-packages/gradio/deprecation.py:40: UserWarning: `optional` parameter is deprecated, and it has no effect
  warnings.warn(value)
/home/pete/Documents/python_code/hugginFace/venv_hugginFace/lib/python3.8/site-packages/gradio/outputs.py:22: UserWarning: Usage of gradio.outputs is deprecated, and will not be supported in the future, please import your components from gradio.components
  warnings.warn(


Running on local URL:  http://127.0.0.1:7872

To create a public link, set `share=True` in `launch()`.
